In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/Telecom_customer churn (100000).csv')

In [ ]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)

change_mou=num_df['change_mou']
change_rev=num_df['change_rev']
num_df=num_df.drop(['change_mou'], axis=1)
num_df=num_df.drop(['change_rev'], axis=1)

num_df=num_df.drop(['Customer_ID'], axis=1)

churn=num_df['churn']
num_df=num_df.drop(['churn'], axis=1)

#no convertion

num_df=num_df.fillna(conData)

num_df[num_df < 0]=0

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

In [ ]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0, random_state=42)
#train=pd.concat([X_train, y_train], axis=1)
#test=pd.concat([X_test, y_test], axis=1)
train=pd.concat([X, y], axis=1)
test=pd.concat([X, y], axis=1)
X_train=X
X_test=X
y_train=y
y_test=y

In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0
    
 
        

opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=10
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=200
opt.enable_privacy=1

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios

array([0.50438, 0.49562])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/checkpoint_GAN_MWOE_privacy_1.pth')

Epoch : 1 Loss D real :  0.3875087958023634 Loss D fake :  0.011936870525085966 Loss G :  0.01132754086818584 Epsilon spent :  1.4604744918270696
Epoch : 2 Loss D real :  0.383797765640979 Loss D fake :  0.010219747594863915 Loss G :  0.010175761699929769 Epsilon spent :  1.4818333005328603
Epoch : 3 Loss D real :  0.38944818624226274 Loss D fake :  0.01 Loss G :  0.010055584390954515 Epsilon spent :  1.5031921092386513
Epoch : 4 Loss D real :  0.3826154452307542 Loss D fake :  0.01 Loss G :  0.01005055244671806 Epsilon spent :  1.5245509179444423
Epoch : 5 Loss D real :  0.38771386390010515 Loss D fake :  0.01 Loss G :  0.01004876100567867 Epsilon spent :  1.5459097266502333
Epoch : 6 Loss D real :  0.3866187578666139 Loss D fake :  0.01 Loss G :  0.010050376444995979 Epsilon spent :  1.567268535356024
Epoch : 7 Loss D real :  0.3784039238134151 Loss D fake :  0.01 Loss G :  0.010049707875652706 Epsilon spent :  1.588627344061815
Epoch : 8 Loss D real :  0.3907707356170249 Loss D fake

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
    syn_data = model.generate(X_train.shape[0], class_ratios)
    X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

In [ ]:
X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_100percent.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_100percent.csv")

In [ ]:
# Creating downstream learners
learners = []
if opt.downstream_task == "classification":
    names = ['LR', 'Random Forest', 'Neural Network', 'GaussianNB', 'GradientBoostingClassifier', 'DT', 'KNN', 'SVN']

    learners.append((LogisticRegression()))
    learners.append((RandomForestClassifier()))
    learners.append((MLPClassifier(early_stopping=True)))
    learners.append((GaussianNB()))
    learners.append((GradientBoostingClassifier()))
    learners.append(DecisionTreeClassifier())
    learners.append((KNeighborsClassifier()))
    #learners.append((svm.SVC()))

    print("AUC scores of downstream classifiers on test data : ")
    for i in range(0, len(learners)):
        score = learners[i].fit(X_syn, y_syn)
        #score = learners[i].fit(X_train, y_train)
        pred_probs = learners[i].predict_proba(X_test)
        auc_score = roc_auc_score(y_test, pred_probs[:, 1])
        print('-' * 40)
        print('{0}: {1}'.format(names[i], auc_score))